In [ ]:
import torchtext
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import time
import torch


In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.t).squeeze(1)

        batch.l = batch.l.type(torch.FloatTensor)

        loss = criterion(predictions, batch.l)

        acc = binary_accuracy(predictions, batch.l)
        print('Accuracy:' + str(acc))
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            batch.l = batch.l.type(torch.FloatTensor)

            predictions = model(batch.t).squeeze(1)

            loss = criterion(predictions, batch.l)

            acc = binary_accuracy(predictions, batch.l)
            print('Accuracy:'+ str(acc))
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):

        embedded = self.embedding(text)

        output, hidden = self.rnn(embedded)

        assert torch.equal(output[-1, :, :], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))

In [ ]:
# loading polish word2vect pretrained - if you want to use it, just download the word2vect model
vec = torchtext.vocab.Vectors('nkjp+wiki-lemmas-all-100-cbow-ns.txt', cache='C:\\Users\\Michał\\Desktop\\nkjp+wiki-lemmas-all-100-cbow-ns.txt')

In [ ]:
root = r'' # root to folder with data to analyse
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

fields = {'text': ('t', TEXT), 'label': ('l', LABEL)}

train_data, valid_data, test_data = data.TabularDataset.splits(
                            path = root,
                            train = 'train.json',
                            validation= 'valid.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields
)

MAX_VOCAB_SIZE = 250000
TEXT.build_vocab(train_data,max_size = MAX_VOCAB_SIZE,vectors = vec,unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

BATCH_SIZE = 256

train_iterator,valid_iterator ,test_iterator = data.BucketIterator.splits(
    (train_data,valid_data,test_data),
    sort_key = lambda x: len(train_data.examples),
    sort_within_batch = False,
    batch_size=BATCH_SIZE,
    device=device)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

criterion = criterion.to(device)

N_EPOCHS = 3

best_valid_loss = float('inf')


In [ ]:
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model,valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')